# Newsletter Expert RAG System

Building a production-ready Retrieval-Augmented Generation system for email newsletters.

## Pipeline Overview

1. **Preprocessing**: Extract clean text from HTML emails
2. **Chunking**: Break emails into smaller, semantic pieces
3. **Embedding**: Convert chunks to vectors
4. **Indexing**: Store vectors in Qdrant
5. **Retrieval**: Search for relevant chunks
6. **Generation**: Answer questions using retrieved context

## Setup & Imports

In [ ]:
import os
import glob
from pathlib import Path

# Our custom modules
from src.preprocessor import EmailPreprocessor

# For visualization
import matplotlib.pyplot as plt
import pandas as pd

print("✅ Imports complete!")

## Step 1: Email Preprocessing

Let's start by extracting and cleaning text from our HTML emails.

In [ ]:
# Initialize preprocessor
preprocessor = EmailPreprocessor()

# Path to our email files
emails_dir = "emails/emails_to_html"
email_files = glob.glob(f"{emails_dir}/*.html")

print(f"Found {len(email_files)} email files")
print(f"\nFirst 5 files:")
for f in email_files[:5]:
    print(f"  - {Path(f).name}")

### Test with a Single Email

In [ ]:
# Extract first email
test_file = email_files[0]
email_data = preprocessor.extract_from_html(test_file)

print("="*70)
print("EXTRACTED EMAIL DATA")
print("="*70)
print(f"Source: {Path(email_data['source']).name}")
print(f"Subject: {email_data['subject']}")
print(f"From: {email_data['from']}")
print(f"Date: {email_data['date']}")
print(f"\nBody length: {len(email_data['body_text'])} characters")
print(f"\nBody preview (first 300 chars):")
print(email_data['body_text'][:300])
print("...")

### Process All Emails (Start Small)

Let's process a small batch first to see the results.

In [ ]:
# Process first 10 emails (start small!)
num_emails_to_process = 10

processed_emails = []

for email_file in email_files[:num_emails_to_process]:
    try:
        email_data = preprocessor.extract_from_html(email_file)
        processed_emails.append(email_data)
    except Exception as e:
        print(f"Error processing {email_file}: {e}")

print(f"\n✅ Successfully processed {len(processed_emails)} emails")

### Process All Emails (Optional)

Once you've tested with 10 emails and everything works, uncomment this cell to process all emails.

In [ ]:
processed_emails = []
for email_file in email_files:
    try:
        email_data = preprocessor.extract_from_html(email_file)
        processed_emails.append(email_data)
    except Exception as e:
        print(f"Error processing {Path(email_file).name}: {e}")

print(f"\n✅ Successfully processed {len(processed_emails)} emails")

### Explore the Data

In [ ]:
# Create a DataFrame for easy exploration
df = pd.DataFrame([
    {
        'filename': Path(e['source']).name,
        'subject': e['subject'][:50] + '...' if len(e['subject']) > 50 else e['subject'],
        'from': e['from'][:30] + '...' if len(e['from']) > 30 else e['from'],
        'body_length': len(e['body_text'])
    }
    for e in processed_emails
])

display(df)

### Visualize Email Body Lengths

In [ ]:
# Visualize email body lengths
plt.figure(figsize=(10, 5))
plt.hist(df['body_length'], bins=15, edgecolor='black')
plt.xlabel('Email Body Length (characters)')
plt.ylabel('Number of Emails')
plt.title('Distribution of Email Body Lengths')
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"\nStats:")
print(f"  Average length: {df['body_length'].mean():.0f} characters")
print(f"  Min length: {df['body_length'].min()} characters")
print(f"  Max length: {df['body_length'].max()} characters")

## Step 2: Chunking

Breaking emails into smaller, overlapping chunks for better retrieval.

**Why chunking?**
- Embedding models have token limits
- Smaller chunks = more precise retrieval
- Better context for the LLM
- Overlap ensures we don't lose context at boundaries

### Initialize Chunker

In [ ]:
# Import and initialize the chunker
from src.chunker import TokenChunker

# Initialize with default settings (300 tokens per chunk, 50 overlap)
chunker = TokenChunker(chunk_size=300, overlap=50)

print("✅ Chunker initialized!")
print(f"Chunk size: {chunker.chunk_size} tokens")
print(f"Overlap: {chunker.overlap} tokens")

### Test Chunking on Single Email

In [ ]:
# Test with the first email
test_email = processed_emails[0]
chunks = chunker.chunk_email(test_email)

print(f"Email: {test_email['subject']}")
print(f"Body length: {len(test_email['body_text'])} characters")
print(f"Created {len(chunks)} chunk(s)")

# Show first chunk details
if chunks:
    print(f"\nFirst chunk:")
    print(f"  Token count: {chunks[0]['token_count']}")
    print(f"  Chunk ID: {Path(chunks[0]['chunk_id']).name}")
    print(f"  Text preview: {chunks[0]['text'][:200]}...")

### Chunk All Emails

In [ ]:
# Chunk all processed emails
all_chunks = chunker.chunk_multiple_emails(processed_emails)

print(f"Total emails: {len(processed_emails)}")
print(f"Total chunks created: {len(all_chunks)}")
print(f"Average chunks per email: {len(all_chunks) / len(processed_emails):.1f}")

### Explore Chunk Data

In [ ]:
# Create DataFrame for exploration
chunk_df = pd.DataFrame([
    {
        'email': Path(c['source']).name[:40],
        'chunk_index': c['chunk_index'],
        'token_count': c['token_count'],
        'subject': c['subject'][:40] + '...' if len(c['subject']) > 40 else c['subject'],
        'text_preview': c['text'][:60] + '...'
    }
    for c in all_chunks
])

print(f"Showing first 10 chunks:\n")
display(chunk_df.head(10))

### Visualize Chunk Distribution

In [ ]:
# Visualize token counts
plt.figure(figsize=(10, 5))
token_counts = [c['token_count'] for c in all_chunks]
plt.hist(token_counts, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Chunk Size (tokens)')
plt.ylabel('Number of Chunks')
plt.title('Distribution of Chunk Token Counts')
plt.axvline(x=300, color='r', linestyle='--', linewidth=2, label='Max chunk size (300)')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"\nChunk Statistics:")
print(f"  Average tokens per chunk: {sum(token_counts) / len(token_counts):.1f}")
print(f"  Min tokens: {min(token_counts)}")
print(f"  Max tokens: {max(token_counts)}")

## Step 3: Vector Database Setup

Setting up **Qdrant** (Docker) to store our vector embeddings.

**What's happening:**
- Connect to Qdrant running in Docker
- Define a "collection" to store email chunks
- Each chunk will become a vector (embedding) + metadata
- **Data persists** across notebook restarts!

**Why Qdrant?**
- Fast similarity search
- Easy to use
- Perfect for learning RAG
- Can be shared with MCP servers!

**Prerequisites:**
```bash
# Start Qdrant (already running for you):
docker run -d -p 6333:6333 -p 6334:6334 --name qdrant-newsletter qdrant/qdrant:latest
```

**Qdrant Dashboard:** http://localhost:6333/dashboard

### Initialize Qdrant Client (Docker)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# Connect to Qdrant running in Docker
client = QdrantClient(host="localhost", port=6333)

print("✅ Qdrant client initialized (Docker mode)")
print("   Connected to: http://localhost:6333")
print("   Data persists across notebook restarts!")
print("   Dashboard: http://localhost:6333/dashboard")

### Create Collection

A **collection** is like a table in a database. We'll store all our email chunk embeddings here.

**Key settings:**
- **Vector size**: 1536 (OpenAI's `text-embedding-3-small` model dimension)
- **Distance metric**: Cosine (measures similarity between vectors)

In [ ]:
# Collection name
COLLECTION_NAME = "newsletter_chunks"

# Vector configuration
# OpenAI's text-embedding-3-small produces 1536-dimensional vectors
VECTOR_SIZE = 1536

# Create the collection
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=VECTOR_SIZE,
        distance=Distance.COSINE  # Cosine similarity for text
    )
)

print(f"✅ Collection '{COLLECTION_NAME}' created")
print(f"   Vector size: {VECTOR_SIZE}")
print(f"   Distance metric: Cosine")

### Verify Setup

In [ ]:
# Check collection info
collection_info = client.get_collection(COLLECTION_NAME)

print(f"Collection: {COLLECTION_NAME}")
print(f"Vectors: {collection_info.points_count}")
print(f"Status: {collection_info.status}")
print(f"\n✅ Qdrant is ready to receive embeddings!")

## Step 4: Embeddings & Indexing

Converting text chunks to vectors and storing them in Qdrant.

**The process:**
1. Use OpenAI's `text-embedding-3-small` model (1536 dimensions)
2. Create embeddings for all 193 chunks
3. Store vectors + metadata (subject, from, date, chunk ID) in Qdrant

**Why embeddings?**
- Transforms text into numbers that capture semantic meaning
- Similar text = similar vectors (close in vector space)
- Enables fast similarity search

### Import OpenAI Client

In [ ]:
from openai import OpenAI
from qdrant_client.models import PointStruct
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✅ OpenAI client initialized")
print(f"   Ready to create embeddings with text-embedding-3-small")

### Create Embeddings for All Chunks

This will:
- Take each chunk's text
- Send it to OpenAI's embedding API
- Get back a 1536-dimensional vector
- Store it in Qdrant with metadata

**Note:** This costs tokens! ~193 chunks × ~165 tokens/chunk = ~32k tokens ≈ $0.001

In [ ]:
from tqdm import tqdm

# Batch size for embedding API calls
BATCH_SIZE = 100

# Track progress
points_to_upload = []
total_chunks = len(all_chunks)

print(f"Creating embeddings for {total_chunks} chunks...")
print(f"This will use ~{total_chunks * 165} tokens (estimated)")

# Process chunks in batches
for i in tqdm(range(0, total_chunks, BATCH_SIZE)):
    batch = all_chunks[i:i + BATCH_SIZE]
    
    # Extract text from batch
    texts = [chunk['text'] for chunk in batch]
    
    # Create embeddings
    response = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    
    # Prepare points for Qdrant
    for j, chunk in enumerate(batch):
        embedding = response.data[j].embedding
        
        # Create a point with vector and metadata
        point = PointStruct(
            id=i + j,  # Unique ID
            vector=embedding,
            payload={
                "text": chunk["text"],
                "source": chunk["source"],
                "subject": chunk["subject"],
                "from": chunk["from"],
                "date": chunk["date"],
                "chunk_id": chunk["chunk_id"],
                "chunk_index": chunk["chunk_index"],
                "token_count": chunk["token_count"]
            }
        )
        points_to_upload.append(point)

print(f"\n✅ Created {len(points_to_upload)} embeddings")

### Upload to Qdrant

In [ ]:
# Upload all points to Qdrant
client.upsert(
    collection_name=COLLECTION_NAME,
    points=points_to_upload
)

print(f"✅ Uploaded {len(points_to_upload)} vectors to Qdrant")

# Verify upload
collection_info = client.get_collection(COLLECTION_NAME)
print(f"\nCollection now has {collection_info.points_count} vectors")
print("🎉 Indexing complete! Ready for retrieval.")

## Step 5: Retrieval & Generation

The RAG pipeline in action!

**The flow:**
1. **User asks a question** (e.g., "What newsletters did I get about AI?")
2. **Embed the question** using OpenAI (same model as chunks)
3. **Search Qdrant** for top-K most similar chunks (vector similarity)
4. **Retrieve context** from those chunks
5. **Send to LLM** with the question as a prompt
6. **Generate answer** based on retrieved context

This is the "Retrieval-Augmented Generation" part!

### Helper Function: Retrieve Similar Chunks

In [ ]:
def retrieve_similar_chunks(query: str, top_k: int = 5):
    """
    Find the most relevant chunks for a given query.
    
    Args:
        query: User's question
        top_k: Number of similar chunks to retrieve
        
    Returns:
        List of (chunk_text, metadata) tuples
    """
    # 1. Embed the query using OpenAI
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=query
    ).data[0].embedding
    
    # 2. Search Qdrant for similar vectors
    search_results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_embedding,
        limit=top_k
    ).points
    
    # 3. Extract chunks and metadata
    results = []
    for hit in search_results:
        results.append({
            "text": hit.payload["text"],
            "subject": hit.payload["subject"],
            "from": hit.payload["from"],
            "date": hit.payload["date"],
            "score": hit.score,  # Similarity score (0-1, higher is better)
            "source": hit.payload["source"]
        })
    
    return results

print("✅ Retrieval function ready!")

### Test Retrieval (Without LLM)

In [ ]:
# Test with a sample query
test_query = "Any emails from Instagram?"

print(f"Query: {test_query}\n")
print("=" * 80)

results = retrieve_similar_chunks(test_query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"\n📧 Result {i} (score: {result['score']:.3f})")
    print(f"   Subject: {result['subject']}")
    print(f"   From: {result['from']}")
    print(f"   Date: {result['date']}")
    print(f"   Text preview: {result['text'][:200]}...")
    print("-" * 80)

### Helper Function: Generate Answer with LLM

In [ ]:
def generate_answer(query: str, retrieved_chunks: list):
    """
    Generate an answer using retrieved chunks as context.
    
    Args:
        query: User's question
        retrieved_chunks: List of relevant chunks from retrieval
        
    Returns:
        Generated answer string
    """
    # Build context from retrieved chunks
    context_parts = []
    for i, chunk in enumerate(retrieved_chunks, 1):
        context_parts.append(
            f"[Source {i}] {chunk['subject']} (from {chunk['from']}, {chunk['date']})\n"
            f"{chunk['text']}\n"
        )
    
    context = "\n---\n".join(context_parts)
    
    # Create the prompt
    prompt = f"""You are a helpful assistant that answers questions about newsletter emails.

Use the following context from the user's newsletters to answer their question.
If the context doesn't contain relevant information, say so honestly.

CONTEXT:
{context}

QUESTION: {query}

ANSWER:"""
    
    # Call OpenAI
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions about newsletter emails based on provided context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    
    return response.choices[0].message.content

print("✅ Generation function ready!")

### Complete RAG Pipeline Function

In [ ]:
def ask_newsletter_expert(query: str, top_k: int = 5, show_sources: bool = True):
    """
    Complete RAG pipeline: retrieve + generate.
    
    Args:
        query: User's question
        top_k: Number of chunks to retrieve
        show_sources: Whether to display retrieved sources
        
    Returns:
        Generated answer
    """
    print(f"❓ Question: {query}\n")
    print("🔍 Retrieving relevant chunks...")
    
    # Step 1: Retrieve
    chunks = retrieve_similar_chunks(query, top_k=top_k)
    
    if show_sources:
        print(f"\n📚 Found {len(chunks)} relevant chunks:\n")
        for i, chunk in enumerate(chunks, 1):
            print(f"  {i}. {chunk['subject'][:60]}... (score: {chunk['score']:.3f})")
    
    # Step 2: Generate
    print("\n💭 Generating answer...\n")
    answer = generate_answer(query, chunks)
    
    print("=" * 80)
    print("🤖 ANSWER:")
    print("=" * 80)
    print(answer)
    print("=" * 80)
    
    return answer

print("✅ Complete RAG pipeline ready!")

### Test the Complete RAG System! 🚀

In [ ]:
# Test with your query!
answer = ask_newsletter_expert(
    "Is there any instagram emails about ryan reynolds?",
    top_k=3
)

### Try More Queries!

In [ ]:
# Try your own queries here!
# Examples:
# - "Did I get any newsletters about Python programming?"
# - "What job opportunities were mentioned in my newsletters?"
# - "Show me newsletters about web development"
# - "What newsletters did I get from [specific sender]?"

# Uncomment and modify:
# ask_newsletter_expert("Your question here", top_k=5)

---

## Notes & Observations

Use this space to document your findings, observations, and questions as you work through the pipeline.

### Questions:
- 

### Observations:
- 

### Next Steps:
- Implement chunking
- Set up Qdrant
- Test with small dataset first